In [21]:
import os
import pandas as pd
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool, PythonAstREPLTool
from langchain.agents import load_tools, initialize_agent, AgentType
from getpass import getpass
from langchain.agents.openai_functions_agent.base import (

    OpenAIFunctionsAgent

)
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from langchain.agents import AgentExecutor

In [5]:
os.environ['OPENAI_API_KEY'] = getpass("OpenAI Key:")

In [6]:
dset = pd.read_csv("/Users/devam/PycharmProjects/DataScribe/owid-covid-data.csv")
dset

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323761,ZWE,Africa,Zimbabwe,2023-07-01,265524.0,0.0,15.857,5707.0,0.0,0.000,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
323762,ZWE,Africa,Zimbabwe,2023-07-02,265524.0,0.0,15.857,5707.0,0.0,0.000,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
323763,ZWE,Africa,Zimbabwe,2023-07-03,265604.0,80.0,11.429,5709.0,2.0,0.286,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
323764,ZWE,Africa,Zimbabwe,2023-07-04,265604.0,0.0,11.429,5709.0,0.0,0.286,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN


In [22]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613")
memory = ConversationBufferMemory(memory_key="chat_history")
dataset = {'df': dset}
tools = [PythonAstREPLTool(locals=dataset)]
tool_names = [tool.name for tool in tools]
prompt = OpenAIFunctionsAgent.create_prompt(system_message=SystemMessage(content='You are an intelligent chatbot capable of running Python code to do dataset exploration and machine learning. You are allowed to use external libraries.'))
agent = AgentExecutor.from_agent_and_tools(
    
    agent=OpenAIFunctionsAgent(llm=llm, prompt=prompt, tools=tools, verbose=True),
    tools=tools, 
    verbose=True,
    memory=memory
                                           
)


In [25]:
agent.run("from the dataset /Users/devam/PycharmProjects/DataScribe/owid-covid-data.csv, please plot maximum hospital visits versus new cases smoothed")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\nimport matplotlib.pyplot as plt\n\n# Load the dataset\ndata = pd.read_csv('/Users/devam/PycharmProjects/DataScribe/owid-covid-data.csv')\n\n# Filter the data for relevant columns\nfiltered_data = data[['date', 'new_cases_smoothed', 'hosp_patients_max']].dropna()\n\n# Plot the data\nplt.figure(figsize=(10, 6))\nplt.plot(filtered_data['date'], filtered_data['new_cases_smoothed'], label='New Cases Smoothed')\nplt.plot(filtered_data['date'], filtered_data['hosp_patients_max'], label='Maximum Hospital Visits')\nplt.xlabel('Date')\nplt.ylabel('Count')\nplt.title('Maximum Hospital Visits vs New Cases Smoothed')\nplt.legend()\nplt.xticks(rotation=45)\nplt.show()\n"}`


KeyError: "['hosp_patients_max'] not in index"It seems that the column name "hosp_patients_max" does not exist in the dataset. Could you please provide the correct column name for maximum hospital visits?

> Finished chain.

'It seems that the column name "hosp_patients_max" does not exist in the dataset. Could you please provide the correct column name for maximum hospital visits?'